# Large Rocks Detection Project

### Outline

1. Dataset provided store in train/val/test (AND NORMALISED?)
2. Data Augmentation
3. load YOLOv8 pre-trained model
3. train the model on the training set
4. test on the validation set
5. fight overfitting with regularization